In [1]:
#########################################################################
# Script to parse alignment and remove range of columns via excel file  #
# Jack G. Payette, MIT Fournier Lab, EAPS.                              #
# MIT Software License                                                  #
# E-mail: payette@mit.edu                                               #
#########################################################################
from Bio import AlignIO
from Bio import SeqIO
import pandas as pd
import itertools
import re
import os

In [2]:
# Read fasta alignment file into python ~ these variables could be converted to arg_parse for standalone script
alignment_name='150fastas.fasta'
alignment_format='fasta'
alignment = AlignIO.read(alignment_name,alignment_format)

In [3]:
# Print AlignIO object
print(alignment)

SingleLetterAlphabet() alignment with 150 rows and 57151 columns
-MQTDL--SNSSLFNHKSVMTDEILVSI-DHFPL---IQDNQLR...--- AG-311-I09
--------------------------------------------...END AG-315-C08
--------------------------------------------...--- AG-315-C22
--------------------------------------------...GND AG-315-J23
--------------------------------------------...GND AG-315-K21
-MQTDL--SDSSLFNHKSVMTDEILFSV-DKYPL---ISNNKLT...--- AG-321-G20
-MKEGPI-SSCSNFNHVPIMGKEIIQSL-KELPS---ELTKQGL...--- AG-341-I22
-MQTDL--SDSSFFNHKSVMTDEIIASL-EHFPS---INNKQLK...AS- AG-347-I15
-MQTDL--SDSSLFNHKSVMTDEILVSV-DQYPL---ISGKKLT...AS- AG-347-J22
-MQTDL--SDSSFFNHKSVMTDEIMASL-RHYPL---IDNNQLK...AS- AG-347-K18
--------------------------------------------...--- AG-347-N23
-MEDQPS-LSSSDVLHQPVLVEQVLNQF-FTLPS---DLLDGGI...GND AG-363-A06
-MPNPSS-VNASNFHHLPVLANDVMQSI-NNLPT---KLLDGGL...SDD AG-363-A16
-MPEQFP-ADASGFHHVPVLSNELLNFT-EALPS---DLLDGGL...--- AG-363-B04
-MKEESK-IVTSKFKHTPVLSREVIQAI-SQLPA---TLLKKGT...--- AG-363-B11
-----

In [4]:
# Print ID of each row ~ taxa name
# Print length of columns ~ sites in alignment
for record in alignment:
    print("%s %i" % (record.id, len(record)))

AG-311-I09 57151
AG-315-C08 57151
AG-315-C22 57151
AG-315-J23 57151
AG-315-K21 57151
AG-321-G20 57151
AG-341-I22 57151
AG-347-I15 57151
AG-347-J22 57151
AG-347-K18 57151
AG-347-N23 57151
AG-363-A06 57151
AG-363-A16 57151
AG-363-B04 57151
AG-363-B11 57151
AG-363-C02 57151
AG-363-C20 57151
AG-363-G23 57151
AG-363-I04 57151
AG-363-J23 57151
AG-363-K07 57151
AG-363-L02 57151
AG-363-L17 57151
AG-363-L19 57151
AG-363-M17 57151
AG-363-M20 57151
AG-363-M21 57151
AG-363-N03 57151
AG-363-N16 57151
AG-363-N20 57151
AG-363-O06 57151
AG-363-O15 57151
AG-363-O16 57151
AG-363-O21 57151
AG-363-P01 57151
AG-363-P08 57151
AG-363-P15 57151
AG-363-P19 57151
AG-388-D03 57151
AG-402-B03 57151
AG-402-B19 57151
AG-402-G22 57151
AG-402-I21 57151
AG-402-K05 57151
AG-402-K14 57151
AG-402-K21 57151
AG-402-L18 57151
AG-402-M23 57151
AG-402-N10 57151
AG-402-N21 57151
AG-402-P16 57151
AG-402-P18 57151
AG-409-A19 57151
AG-409-A23 57151
AG-409-B05 57151
AG-409-B13 57151
AG-409-B20 57151
AG-409-C21 57151
AG-409-D09 571

In [5]:
# Get alignment length using AlignIO method and store as variable
alignment_length = alignment.get_alignment_length()

In [6]:
print(alignment_length)

57151


In [7]:
# Calculate and print the last possible slice in the alignment, given python ZERO indexing
print(alignment_length-1)

57150


In [8]:
#print the last column position of all taxa/rows
print(alignment[::,alignment_length-1])

-D-DD------DD--DDDDDDD----D-DDDDD--D---DD-D-D--D-DD-DDDDD-EDDDDDD-DDDDDDDD--DDD------EDD--D-----DD-D------DDDDDDDDD-D-------D--DDDDGDDDDDGDDDGDDDDDGGG


In [9]:
### Import excel file from Greg for gappy Areas to remove
# '-' is the delimiter for the column range to DELETE (Inclusive) from the Alignment
# Keep everything EXCEPT columns in these RANGES
#~ these variables could be converted to arg_parse for standalone script
excel_file_name = 'gappyAreas.xlsx'

In [10]:
# Rename column from: 'regions <75% present (deleted)'' to 'Range'
cols_to_remove = pd.read_excel(excel_file_name,names=['Range'])

In [11]:
cols_to_remove

,Range
0,56609-56939
1,51160-54379
2,47959-50570
3,45350-47816
4,42402-44768
5,40270-42094
6,39171-39525
7,38116-39005
8,35611-37580
9,34844-34969


In [12]:
cols_to_remove[23:] = '1-45' #correct typo from '1--45' to be '1-45'

In [13]:
cols_to_remove.tail()

,Range
19,2124-3563
20,1568-2007
21,702-1436
22,313-342
23,1-45


In [14]:
# PARSE THE RANGE DATA MANUALLY
# !!! Subtract 1 from each column position for PYTHON ZERO INDEX !!!
# Print the first column minus 1 and the second column plus 1, of the given columns to delete
# These are the slices (ignoring -1 since the slice starts after the first column)
# I know... this isn't a great way, but works...
for each in cols_to_remove.Range.iloc[::-1]:
    print(int(each.split('-')[0])-1-1)
    print(int(each.split('-')[1])+1-1)

#manually print the last column position, since the columns to remove, don't go all the way to the end!!!
print(alignment_length-1)

-1
45
311
342
700
1436
1566
2007
2122
3563
3821
4254
4412
11827
12355
13426
13503
13585
13922
14161
14417
14895
15883
22376
22991
26920
27654
34766
34842
34969
35609
37580
38114
39005
39169
39525
40268
42094
42400
44768
45348
47816
47957
50570
51158
54379
56607
56939
57150


In [15]:
# MANUALLY ASSEMBLE ALIGNMENT
# Concatenate the slices not in the SET (RANGE) of those to remove, and keep those
# Lookout for Python zero indexing
# ~taking the column AFTER the first in range and taking the column BEFORE the next in the range
alignment_out = \
alignment[:,45:311]     +alignment[:,342:700]    +alignment[:,1436:1566]  +alignment[:,2007:2122]+\
alignment[:,3563:3821]  +alignment[:,4254:4412]  +alignment[:,11827:12355]+alignment[:,13426:13503]+\
alignment[:,13585:13922]+alignment[:,14161:14417]+alignment[:,14895:15883]+alignment[:,22376:22991]+\
alignment[:,26920:27654]+alignment[:,34766:34842]+alignment[:,34969:35609]+alignment[:,37580:38114]+\
alignment[:,39005:39169]+alignment[:,39525:40268]+alignment[:,42094:42400]+alignment[:,44768:45348]+\
alignment[:,47816:47957]+alignment[:,50570:51158]+alignment[:,54379:56607]+alignment[:,56939:57150]

In [16]:
# WRITE OUT TRIMMED ALIGNMENT ~ these variables could be converted to arg_parse for standalone script
with open("alignment_out.faa", "w") as handle:
    count = SeqIO.write(alignment_out, handle, "fasta")

In [17]:
### AUTOMATIC ASSEMBLY ###

In [18]:
# Generate list of pairs of columns for range to delete
# Sort pandas dataframe in reverse
# Skip first item if <0
pairs=[]
for each in cols_to_remove.Range.iloc[::-1]:
    pair=[(int(each.split('-')[0])-1-1),(int(each.split('-')[1])+1-1)]
    if int(pair[0]) > 0 !=True:
        pairs.append(pair[0])
    if int(pair[1]) > 0 !=True:
        pairs.append(pair[1])     
    else:
        continue

In [19]:
print(len(pairs))
print(pairs)

47
[45, 311, 342, 700, 1436, 1566, 2007, 2122, 3563, 3821, 4254, 4412, 11827, 12355, 13426, 13503, 13585, 13922, 14161, 14417, 14895, 15883, 22376, 22991, 26920, 27654, 34766, 34842, 34969, 35609, 37580, 38114, 39005, 39169, 39525, 40268, 42094, 42400, 44768, 45348, 47816, 47957, 50570, 51158, 54379, 56607, 56939]


In [20]:
# SUPER IMPORTANT TO ADD THE LAST COLUMN POSITION, since the slice to keep should go up to this.
#
if (len(pairs) == len(cols_to_remove)*2) == False:
    pairs.append(alignment_length-1)

In [21]:
print(len(pairs))
print(pairs)

48
[45, 311, 342, 700, 1436, 1566, 2007, 2122, 3563, 3821, 4254, 4412, 11827, 12355, 13426, 13503, 13585, 13922, 14161, 14417, 14895, 15883, 22376, 22991, 26920, 27654, 34766, 34842, 34969, 35609, 37580, 38114, 39005, 39169, 39525, 40268, 42094, 42400, 44768, 45348, 47816, 47957, 50570, 51158, 54379, 56607, 56939, 57150]


In [22]:
#Build new EMPTY alignment from original
alignment_new = alignment[:,0:0]
alignment_new

#Define pairwise iterable
def pairwise(iterable):
    a = iter(iterable)
    return zip(a, a)

# Iterate over pairs and concatenate slices together, based on pairs
for x, y in pairwise(pairs):
    alignment_new += alignment[:,x:y] 

In [23]:
print(alignment_new)

SingleLetterAlphabet() alignment with 150 rows and 11031 columns
IDATLGGGGHSYQLLKKYPDLKIIGLDHDPFARKSAFSKLKEFK...--- AG-311-I09
--------------------------------------------...YEN AG-315-C08
--------------------------------------------...--- AG-315-C22
--------------------------------------------...YGN AG-315-J23
--------------------------------------------...YGN AG-315-K21
IDATLGGGGHSYQLLKKYPDLKIIGLDHDPFARKSAFSKLEEFK...--- AG-321-G20
IDATIGGGGHSAQILENFPGIKIIGLDQDPIAREAASKKLIKFG...--- AG-341-I22
IDATLGGGGHSYHLLRKYSDLNIIGIDQDPFARKSASNKLDEFK...YAS AG-347-I15
IDATLGGGGHSYQLLKKYPDLKIIGLDHDPYARKSALSKLEEFK...YAS AG-347-J22
IDATLGGGGHSYYLLRKYSDLNIIGLDQDPFARKSASKKLDEFK...YAS AG-347-K18
--------------------------------------------...--- AG-347-N23
VDATLGYGGHASALLKAFPGLRLIGIDQDPVARSFAKARLSCFG...YGN AG-363-A06
IDATLGGGGHCAMLLEAHPNIHAIGLDQDSSARKAAQKRLSRFG...YSD AG-363-A16
IDATVGGGGHGETLLKAYPQLRLIGLDQDAFAREAASKKLSNFG...--- AG-363-B04
IDATLGGGGHASLALEAYPNLHIIGLDHDPAAISAATDQLNIYG...--- AG-363-B11
-----

In [24]:
# WRITE OUT TRIMMED ALIGNMENT ~ these variables could be converted to arg_parse for standalone script
with open(alignment_name+'_trimmed', "w") as handle:
    count = SeqIO.write(alignment_new, handle, "fasta")

In [25]:
print('Same rows:',len(alignment_new) == len(alignment))
print('Different length:',alignment_new.get_alignment_length() == alignment.get_alignment_length())

Same rows: True
Different length: False


In [26]:
### Compare alignment_out from manual assembly to alignment_new from auto assembly

In [27]:
len(alignment_out) == len(alignment_new)

True

In [28]:
alignment_out.get_alignment_length() == alignment_new.get_alignment_length()

True

In [29]:
### FIN ###